In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [8]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_28,ohe_29,ohe_30,ohe_31,ohe_32,ohe_33,ohe_34,ohe_35,ohe_36,ohe_37
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
                'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS']),           
                  'loss_function': 'RMSE',        
                  'eval_metric': 'RMSE',               
                  'max_depth':trial.suggest_int('max_depth', 3, 5),           
                  'learning_rate':trial.suggest_uniform('learning_rate', 9.1e-3, 1.11e-2),        
                  'n_estimators':N_ESTIMATORS,        
                  'max_bin':trial.suggest_int('max_bin', 216, 316),           
                  'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 23, 43),    
                  'reg_lambda':trial.suggest_uniform('reg_lambda', 0.609, 0.809),      
                  'subsample': trial.suggest_uniform('subsample', 0.631, 0.731)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model = ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-27 06:04:07,381] A new study created in memory with name: no-name-d67f374a-f279-4631-b190-900d36148ef2


0:	learn: 0.7471592	test: 0.7435669	best: 0.7435669 (0)	total: 80.3ms	remaining: 26m 45s
1000:	learn: 0.7296964	test: 0.7270721	best: 0.7270721 (1000)	total: 20.2s	remaining: 6m 23s
2000:	learn: 0.7247457	test: 0.7232345	best: 0.7232345 (2000)	total: 39.5s	remaining: 5m 54s
3000:	learn: 0.7215558	test: 0.7211757	best: 0.7211757 (3000)	total: 58.5s	remaining: 5m 31s
4000:	learn: 0.7190725	test: 0.7198923	best: 0.7198923 (4000)	total: 1m 17s	remaining: 5m 10s
5000:	learn: 0.7169740	test: 0.7190800	best: 0.7190800 (5000)	total: 1m 36s	remaining: 4m 50s
6000:	learn: 0.7150866	test: 0.7184849	best: 0.7184849 (6000)	total: 1m 55s	remaining: 4m 30s
7000:	learn: 0.7133393	test: 0.7180658	best: 0.7180658 (7000)	total: 2m 15s	remaining: 4m 11s
8000:	learn: 0.7116865	test: 0.7177324	best: 0.7177324 (7987)	total: 2m 34s	remaining: 3m 51s
9000:	learn: 0.7101085	test: 0.7174777	best: 0.7174777 (9000)	total: 2m 53s	remaining: 3m 31s
10000:	learn: 0.7085945	test: 0.7172985	best: 0.7172985 (10000)	tota

[I 2021-08-27 06:14:13,343] Trial 0 finished with value: 0.7165666689294097 and parameters: {'bootstrap_type': 'Bernoulli', 'max_depth': 5, 'learning_rate': 0.01072504040351985, 'max_bin': 262, 'min_data_in_leaf': 39, 'reg_lambda': 0.7780723083893594, 'subsample': 0.6520074389163234}. Best is trial 0 with value: 0.7165666689294097.


0:	learn: 0.7471700	test: 0.7435786	best: 0.7435786 (0)	total: 21.4ms	remaining: 7m 7s
1000:	learn: 0.7310595	test: 0.7279993	best: 0.7279993 (1000)	total: 17.5s	remaining: 5m 31s
2000:	learn: 0.7266254	test: 0.7243070	best: 0.7243070 (2000)	total: 34.6s	remaining: 5m 10s
3000:	learn: 0.7238065	test: 0.7221782	best: 0.7221782 (3000)	total: 52s	remaining: 4m 54s
4000:	learn: 0.7216764	test: 0.7207274	best: 0.7207274 (4000)	total: 1m 9s	remaining: 4m 35s
5000:	learn: 0.7199237	test: 0.7196891	best: 0.7196891 (5000)	total: 1m 26s	remaining: 4m 18s
6000:	learn: 0.7184534	test: 0.7189637	best: 0.7189637 (6000)	total: 1m 42s	remaining: 3m 59s
7000:	learn: 0.7171465	test: 0.7184039	best: 0.7184039 (7000)	total: 1m 59s	remaining: 3m 42s
8000:	learn: 0.7159429	test: 0.7179644	best: 0.7179644 (8000)	total: 2m 16s	remaining: 3m 25s
9000:	learn: 0.7148420	test: 0.7176600	best: 0.7176592 (8997)	total: 2m 33s	remaining: 3m 8s
10000:	learn: 0.7137889	test: 0.7173847	best: 0.7173847 (9999)	total: 2m 5

[I 2021-08-27 06:25:24,006] Trial 1 finished with value: 0.7161769042316662 and parameters: {'bootstrap_type': 'Bernoulli', 'max_depth': 4, 'learning_rate': 0.010772341636327353, 'max_bin': 272, 'min_data_in_leaf': 23, 'reg_lambda': 0.7018984802771335, 'subsample': 0.6970525758346275}. Best is trial 1 with value: 0.7161769042316662.


...

0:	learn: 0.7471602	test: 0.7435735	best: 0.7435735 (0)	total: 21.6ms	remaining: 7m 12s
1000:	learn: 0.7308416	test: 0.7278294	best: 0.7278294 (1000)	total: 20.4s	remaining: 6m 27s
2000:	learn: 0.7263350	test: 0.7241878	best: 0.7241878 (2000)	total: 40.7s	remaining: 6m 5s
3000:	learn: 0.7233854	test: 0.7219650	best: 0.7219650 (3000)	total: 1m	remaining: 5m 44s
4000:	learn: 0.7210961	test: 0.7203817	best: 0.7203817 (4000)	total: 1m 21s	remaining: 5m 24s
5000:	learn: 0.7192746	test: 0.7193038	best: 0.7193038 (5000)	total: 1m 41s	remaining: 5m 4s
6000:	learn: 0.7177049	test: 0.7185454	best: 0.7185452 (5999)	total: 2m 1s	remaining: 4m 44s
7000:	learn: 0.7162915	test: 0.7179743	best: 0.7179743 (7000)	total: 2m 22s	remaining: 4m 24s
8000:	learn: 0.7149711	test: 0.7175266	best: 0.7175262 (7999)	total: 2m 42s	remaining: 4m 4s
9000:	learn: 0.7137233	test: 0.7171890	best: 0.7171890 (9000)	total: 3m 3s	remaining: 3m 43s
10000:	learn: 0.7125514	test: 0.7169164	best: 0.7169164 (10000)	total: 3m 23s

[I 2021-08-27 11:54:43,376] Trial 28 finished with value: 0.7159349931889931 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 4, 'learning_rate': 0.010910896020225032, 'max_bin': 279, 'min_data_in_leaf': 41, 'reg_lambda': 0.6333543721229331, 'subsample': 0.690596102603122}. Best is trial 17 with value: 0.7157891524889244.


0:	learn: 0.7471514	test: 0.7435647	best: 0.7435647 (0)	total: 26.1ms	remaining: 8m 42s
1000:	learn: 0.7297934	test: 0.7271409	best: 0.7271409 (1000)	total: 23.5s	remaining: 7m 26s
2000:	learn: 0.7248760	test: 0.7233477	best: 0.7233477 (2000)	total: 47.1s	remaining: 7m 3s
3000:	learn: 0.7214992	test: 0.7210553	best: 0.7210553 (3000)	total: 1m 10s	remaining: 6m 39s
4000:	learn: 0.7188939	test: 0.7196633	best: 0.7196633 (4000)	total: 1m 34s	remaining: 6m 17s
5000:	learn: 0.7166745	test: 0.7187209	best: 0.7187209 (5000)	total: 1m 57s	remaining: 5m 53s
6000:	learn: 0.7146673	test: 0.7180496	best: 0.7180496 (6000)	total: 2m 21s	remaining: 5m 29s
7000:	learn: 0.7127993	test: 0.7175859	best: 0.7175859 (7000)	total: 2m 44s	remaining: 5m 5s
8000:	learn: 0.7110271	test: 0.7172000	best: 0.7172000 (8000)	total: 3m 8s	remaining: 4m 41s
9000:	learn: 0.7093326	test: 0.7169569	best: 0.7169564 (8998)	total: 3m 31s	remaining: 4m 17s
10000:	learn: 0.7076994	test: 0.7167287	best: 0.7167287 (10000)	total: 

[I 2021-08-27 12:06:23,530] Trial 29 finished with value: 0.7161384375016178 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 5, 'learning_rate': 0.01002971034613261, 'max_bin': 316, 'min_data_in_leaf': 39, 'reg_lambda': 0.6591739149318099, 'subsample': 0.7072701791378665}. Best is trial 17 with value: 0.7157891524889244.


Number of finished trials: 30
Best trial: {'bootstrap_type': 'MVS', 'max_depth': 4, 'learning_rate': 0.010463726141582195, 'max_bin': 308, 'min_data_in_leaf': 41, 'reg_lambda': 0.729088978127274, 'subsample': 0.6661693217916411}


In [14]:
study.best_params

{'bootstrap_type': 'MVS',
 'max_depth': 4,
 'learning_rate': 0.010463726141582195,
 'max_bin': 308,
 'min_data_in_leaf': 41,
 'reg_lambda': 0.729088978127274,
 'subsample': 0.6661693217916411}

# Log

====== Ordinal encoding =========

0.7161554542524746 no noise ver2


====== One-hot encoding =========

0.7160673272565011 no noise ver1

0.7158719075205342 no noise ver3 (narrow space)

